In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.loc[train["Browser_Used"]=="Google Chrome","Browser_Used"]=0
train.loc[train["Browser_Used"]=="Chrome","Browser_Used"]=0
train.loc[train["Browser_Used"]=="InternetExplorer","Browser_Used"]=1
train.loc[train["Browser_Used"]=="IE","Browser_Used"]=1
train.loc[train["Browser_Used"]=="Internet Explorer","Browser_Used"]=1
train.loc[train["Browser_Used"]=="Mozilla Firefox","Browser_Used"]=2
train.loc[train["Browser_Used"]=="Firefox","Browser_Used"]=2
train.loc[train["Browser_Used"]=="Mozilla","Browser_Used"]=2
train.loc[train["Browser_Used"]=="Edge","Browser_Used"]=3
train.loc[train["Browser_Used"]=="Safari","Browser_Used"]=4
train.loc[train["Browser_Used"]=="Opera","Browser_Used"]=5

train["Device_Used"]=train["Device_Used"].astype("category")
train["Device_Used"].cat.categories=[0,1,2]

train["Is_Response"]=train["Is_Response"].astype("category")
train["Is_Response"].cat.categories=[0,1]

In [4]:
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,3,1,1
1,id10327,I stayed at the Crown Plaza April -- - April -...,1,1,1
2,id10328,I booked this hotel through Hotwire at the low...,2,2,1
3,id10329,Stayed here with husband and sons on the way t...,1,0,0
4,id10330,My girlfriends and I stayed here to celebrate ...,3,2,1


In [5]:
description=train.loc[:,"Description"]
vec=TfidfVectorizer(stop_words='english')
print("Conveting document into tf-idf vector")
a=vec.fit_transform(description)
des=a.toarray()

Conveting document into tf-idf vector


In [10]:
brow=np.array(train.loc[:,"Browser_Used"])
dev=np.array(train.loc[:,"Device_Used"])
label=np.array(train.loc[:,"Is_Response"])

In [11]:
clf=MultinomialNB()
eq=clf.fit(des,label)

In [12]:
eq.score(des,label)

0.83851330525017975

In [14]:
test=pd.read_csv("test.csv")
test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [15]:
test.loc[test["Browser_Used"]=="Google Chrome","Browser_Used"]=0
test.loc[test["Browser_Used"]=="Chrome","Browser_Used"]=0
test.loc[test["Browser_Used"]=="InternetExplorer","Browser_Used"]=1
test.loc[test["Browser_Used"]=="IE","Browser_Used"]=1
test.loc[test["Browser_Used"]=="Internet Explorer","Browser_Used"]=1
test.loc[test["Browser_Used"]=="Mozilla Firefox","Browser_Used"]=2
test.loc[test["Browser_Used"]=="Firefox","Browser_Used"]=2
test.loc[test["Browser_Used"]=="Mozilla","Browser_Used"]=2
test.loc[test["Browser_Used"]=="Edge","Browser_Used"]=3
test.loc[test["Browser_Used"]=="Safari","Browser_Used"]=4
test.loc[test["Browser_Used"]=="Opera","Browser_Used"]=5

test["Device_Used"]=test["Device_Used"].astype("category")
test["Device_Used"].cat.categories=[0,1,2]

In [16]:
description_test=test.loc[:,"Description"]
a_test=vec.transform(description_test)
des_test=a_test.toarray()

In [17]:
prediction=eq.predict(des_test)

In [22]:
prediction

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
prediction=np.array(prediction )

In [25]:
loan_id=np.ravel(test.loc[:,['User_ID']])
ans=pd.DataFrame(prediction,loan_id,columns=['Is_Response'])

ans.loc[ans['Is_Response']==1,['Is_Response']]='not_happy'
ans.loc[ans['Is_Response']==0,['Is_Response']]='happy'

ans.to_csv('answer.csv',index_label=['User_ID'])